In [ ]:
import FinanceDataReader as fdr
import pandas as pd
import os
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 나스닥 모든 종목 리스트 가져오기
nasdaq_symbols = fdr.StockListing('NASDAQ')['Symbol'].tolist()
total_stocks = len(nasdaq_symbols)  

# 나스닥 종목 리스트를 순회하면서 처리
for i, symbol in enumerate(nasdaq_symbols, start=1): 
    print(f"Processing {symbol}... ({i}/{total_stocks})")  # 진행 상황

    # 오늘 날짜와 2년 전 날짜 계산
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=730)).strftime('%Y-%m-%d')

    # 1. 해당 종목의 데이터를 가져오기
    try:
        df = fdr.DataReader(symbol, start_date, end_date)

        # 데이터가 비어있지 않은지 확인
        if not df.empty:
            print(f"{symbol} data download complete (example)")
            print(df.head()) 
        else:
            print(f"No data available for {symbol}.")
            continue  # 데이터가 없는 종목은 건너뜀
    except Exception as e:
        print(f"Error occurred while downloading {symbol} data: {e}")
        continue

    # 2. 데이터 전처리
    scaler = MinMaxScaler(feature_range=(0, 1))

    # 데이터 전처리 수행
    if not df.empty:
        close_prices = df['Close'].values.reshape(-1, 1)

        scaled_data = scaler.fit_transform(close_prices)

        df['Scaled_Close'] = scaled_data

        # 전처리된 데이터 출력
        print(f"{symbol} data preprocessing complete (example)")
        print(df[['Close', 'Scaled_Close']].head())  # 원본 종가와 정규화된 데이터 출력

    # 3. 데이터셋 분할: 훈련 데이터와 테스트 데이터
    train_size = int(len(scaled_data) * 0.8)  # 데이터의 80%를 훈련 데이터로 사용
    train_data = scaled_data[:train_size]  
    test_data = scaled_data[train_size:]   

    # 충분한 테스트 데이터가 있는지 확인
    if len(test_data) <= 61:  
        print(f"Not enough test data for {symbol}. Skipping this stock.")
        continue

# 4. GRU 입력 형식에 맞게 데이터 변형
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])  # 입력 데이터
        y.append(data[i + time_step, 0])      # 타겟 데이터
    return np.array(X), np.array(y)

# 5. time step 설정
time_step = 60  
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# 6. GRU 입력 형식에 맞게 데이터를 3D 배열로 변형
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # (samples, time steps, features)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 7. GRU 모델 구축
gru_model = Sequential()  # Sequential 모델 초기화
gru_model.add(Input(shape=(X_train.shape[1], 1)))  # Input 레이어 추가
gru_model.add(GRU(units=50, return_sequences=True))  # 첫 번째 GRU 레이어
gru_model.add(Dropout(0.2))  # 과적합 방지를 위한 Dropout
gru_model.add(GRU(units=50, return_sequences=False))  # 두 번째 GRU 레이어
gru_model.add(Dropout(0.2))  # 과적합 방지를 위한 Dropout
gru_model.add(Dense(units=1))  # 최종 출력 레이어
gru_model.compile(optimizer='adam', loss='mean_squared_error')  # 모델 컴파일

# 모델 훈련
gru_model.fit(X_train, y_train, epochs=100, batch_size=32)

# 8. 예측 수행
gru_predicted_prices = gru_model.predict(X_test)    # GRU 예측

# 예측 결과를 원래 스케일로 변환
gru_predicted_prices = scaler.inverse_transform(gru_predicted_prices)

# 결과 시각화
plt.figure(figsize=(14, 7))
plt.plot(df.index[len(train_data) + time_step + 1:], gru_predicted_prices, color='green', label='GRU Predicted Price')  # GRU 예측 결과
plt.plot(df.index, df['Close'], color='blue', label='Actual Price')  # 실제 가격
plt.title(f'{symbol} Stock Price Prediction using GRU')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

100%|█████████████████████████████████████████████████████████████████████████████| 3693/3693 [00:06<00:00, 582.20it/s]


Processing AAPL... (1/3693)
AAPL data download complete (example)
                  Open        High         Low       Close     Volume  \
2022-10-10  140.419998  141.889999  138.570007  140.419998   74899000   
2022-10-11  139.899994  141.350006  138.220001  138.979996   77033700   
2022-10-12  139.130005  140.360001  138.160004  138.339996   70433700   
2022-10-13  134.990005  143.589996  134.369995  142.990005  113224000   
2022-10-14  144.309998  144.520004  138.190002  138.380005   88598000   

             Adj Close  
2022-10-10  138.881226  
2022-10-11  137.457001  
2022-10-12  136.824005  
2022-10-13  141.423065  
2022-10-14  136.863586  
AAPL data preprocessing complete (example)
                 Close  Scaled_Close
2022-10-10  140.419998      0.140255
2022-10-11  138.979996      0.127140
2022-10-12  138.339996      0.121311
2022-10-13  142.990005      0.163661
2022-10-14  138.380005      0.121676
Processing NVDA... (2/3693)
NVDA data download complete (example)
              